In [2]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

from statsforecast.models import AutoETS, AutoMFLES
from statsforecast.core import StatsForecast
from utilsforecast.evaluation import evaluate
from typing import List

#sales = pd.read_csv("Phase 0 - Sales.csv", na_values=np.nan)
sales = pd.read_csv("filtered_second_file.csv", na_values=np.nan)
df = sales.set_index(["Client", "Warehouse","Product"]).stack().reset_index()
df["unique_id"] = df["Client"].astype(str) + "/" + df["Warehouse"].astype(str) + "/" + df["Product"].astype(str)
df = df.drop(columns =  ["Client", "Warehouse", "Product"])
df.columns = ["ds", "y", "unique_id"]
df["ds"] = pd.to_datetime(df["ds"])

# Model inputs
horizon = 13
freq = "W-MON"

seasonal_period = 52

config = {
    'seasonality_weights': [True, False],  # Allows for shifting seasonalities over time
    'smoother': [True, False],  # Exponential smoothing or simple moving average
    'ma': [seasonal_period, int(seasonal_period / 2), None],  # Moving average order
    'seasonal_period': [None, seasonal_period],  # Seasonal period
}

model = AutoMFLES(season_length=52,
                  test_size=26,
                  n_windows=2,
                  metric='mse',
                  
config=config #We can pass a custom config here
                  )

sf = StatsForecast(
                  models=[model],
                  freq=freq,
                  n_jobs=-1,
                  verbose=True
                          )

# Fit the model, takes about 15 mins to tune and forecast on a basic google colab
sf.fit(df = df)

# Generate test predictions
yhat_test = sf.predict(h=horizon)
yhat_test = yhat_test.reset_index()

# Enforce non-negativity
yhat_test[str(sf.models[0])] = yhat_test[str(sf.models[0])].clip(0)

# We use the name of the model
name = "submitsept27_01.csv"

# Apply some data wrangling to ensure everything is in the expected format of the competition
yhat_test[["Client", "Warehouse", "Product"]] = yhat_test["unique_id"].str.split("/", expand=True)
yhat_test[["Client", "Warehouse", "Product"]] = yhat_test[["Client", "Warehouse", "Product"]].astype(np.int64)
yhat_test = yhat_test.sort_values(by=["Client", "Warehouse", "Product"]).reset_index(drop=True)
yhat_test = yhat_test.drop(columns = "unique_id")
yhat_test = yhat_test.set_index(["Client", "Warehouse", "Product", "ds"])
yhat_test = yhat_test.unstack(3)
yhat_test.columns = yhat_test.columns.get_level_values(1)

yhat_test.to_csv(name)
